# Base Model

A simple multilayer ann for classification task of CVEs - in their vectorization format - into their corresponding CWE. Use of PyThorch library

https://machinelearningmastery.com/building-multilayer-perceptron-models-in-pytorch/

To do:
- Define some hiperparameters (number of layers, activation function, learning rate, batch size) Feito
- Are we gonna use some type of regularization like drop out? Feito
- If the real data is not available, we can see if this model is running with some other simple data COMO?
- Create some kind of visualization of loss function, accuracy, define the type of metric to use, probably F1-score, accuracy (confusion matrix is maybe too big in this scenario) Feito

In [79]:
#!pip install torch

In [80]:
# !pip install Pinecone

In [81]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

Import data

In [82]:
from pinecone import Pinecone

import numpy as np 

In [83]:
pc = Pinecone(api_key="pcsk_3fsLLS_KMLjRJ9jybj773qMy13aawRBFZveGrn2LrZhVP3GtMWet7PdZgNubyB8xRM6JQ7")
index = pc.Index("mc959")

Define hyperparameters

In [84]:
input_size = 768                 # Input vector size (V_CVE_size)
num_classes = 1000               # Number of output classes (N_CWE)
hidden_sizes = [256, 128, 64]    # Sizes of hidden layers
activation_function = nn.ReLU    # Activation function to be used
batch_size = 32                  # Batch size
learning_rate = 1e-3             # Learning rate
num_epochs = 20                  # Number of training epochs
dropout_prob = 0.5               # Dropout probability for regularization

In [85]:
query_vector = vector = np.random.rand(input_size).tolist()

# Query the index for the top 5 most similar vectors
response = index.query(
    vector=query_vector,   # Your query vector with 768 dimensions
    top_k=5,               # Retrieve top 5 closest vectors
    include_values=True,   # Include the vector values in the response
    include_metadata=True, # Include any metadata associated with the vectors
)
    
print(response)

{'matches': [{'id': '7',
              'score': -0.0279348046,
              'values': [-0.0206149295,
                         -0.0910236239,
                         0.0518505834,
                         0.0678608865,
                         0.303363383,
                         -0.385049343,
                         0.121501647,
                         0.303099811,
                         0.163347781,
                         -0.0406463929,
                         -0.0566407107,
                         -0.25614506,
                         -0.180770978,
                         0.0390420109,
                         -0.394991755,
                         -0.0130279213,
                         0.427475095,
                         0.155082524,
                         -0.0375246294,
                         -0.116331071,
                         0.109938607,
                         -0.128310978,
                         -0.310703427,
                         0.430569261,
    

In [86]:
# Specify the ID of the vector you want to retrieve
entry_id = "9"  # The ID of the specific entry you want to retrieve

# Fetch the vector with the specified ID
response = index.fetch(ids=[entry_id])

# Print the response
print(response)

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'9': {'id': '9',
                   'values': [-0.272559613,
                              0.250883,
                              0.0537423752,
                              0.0189300161,
                              0.199495047,
                              -0.393878222,
                              0.0997849554,
                              0.205802023,
                              0.241373897,
                              -0.01333219,
                              -0.0938622653,
                              -0.187647536,
                              0.0105433082,
                              0.00871945824,
                              -0.199283049,
                              0.148712263,
                              0.455019593,
                              0.124796785,
                              0.0763969,
                              -0.0997588784,
                              0.122478947,
           

In [87]:
# Vector size encoder CVE
V_CVE_size = input_size

# Number of possible CWEs to return
N_CWE = num_classes

In [88]:
""" # Model for classification task

model = nn.Sequential()
model.add_module("dense1", nn.Linear(V_CVE_size, 12))
model.add_module("act1", nn.ReLU())
model.add_module("dense2", nn.Linear(12, 8))
model.add_module("act2", nn.ReLU())
model.add_module("output", nn.Linear(8, N_CWE))
model.add_module("outact", nn.Softmax()) """

' # Model for classification task\n\nmodel = nn.Sequential()\nmodel.add_module("dense1", nn.Linear(V_CVE_size, 12))\nmodel.add_module("act1", nn.ReLU())\nmodel.add_module("dense2", nn.Linear(12, 8))\nmodel.add_module("act2", nn.ReLU())\nmodel.add_module("output", nn.Linear(8, N_CWE))\nmodel.add_module("outact", nn.Softmax()) '

In [89]:
""" print(model) """

' print(model) '

In [90]:
# Build the neural network model dynamically
layers = []

# Input layer
layers.append(nn.Linear(input_size, hidden_sizes[0]))
layers.append(activation_function())

# Hidden layers
for i in range(len(hidden_sizes) - 1):
    layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
    layers.append(activation_function())

# Output layer
layers.append(nn.Linear(hidden_sizes[-1], num_classes))
layers.append(nn.Softmax(dim=1))

# Create the sequential model
model = nn.Sequential(*layers)

In [91]:
print("Neural Network Model:")
print(model)

Neural Network Model:
Sequential(
  (0): Linear(in_features=768, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=1000, bias=True)
  (7): Softmax(dim=1)
)


In [92]:
# Loss function definiton

loss_fn = nn.CrossEntropyLoss()
# loss = loss_fn(output, label)

# Defining the optimization method
optimizer = torch.optim.Adam(model.parameters(), lr=Learning_rate)

In [96]:
def fetch_data_from_pinecone(index, num_samples):
    X_data = []
    y_data = []
    namespace = "ns1"
    # Loop until we have collected the desired number of samples
    while len(X_data) < num_samples:
        # Generate a random query vector (or use a specific vector if needed)
        query_vector = np.random.rand(input_size).tolist()
        response = index.query(
            namespace=namespace,
            vector=query_vector,
            top_k=5,  # Adjust as needed to fetch more matches per query
            include_values=True,
            include_metadata=True,
            # Add any filters if necessary
            # filter={"genre": {"$eq": "action"}}
        )
        if response and 'matches' in response:
            for match in response['matches']:
                vector = match.get('values', [])
                metadata = match.get('metadata', {})
                if vector and metadata:
                    X_data.append(vector)
                    y_data.append(metadata.get('label', 0))  # Assuming label is in 'label'
                    # Stop if we've collected enough samples
                    if len(X_data) >= num_samples:
                        break
    return np.array(X_data), np.array(y_data)

In [94]:
num_samples = 10

In [ ]:
# Get the data
X_array, y_array = fetch_data_from_pinecone(index, num_samples)

In [ ]:
# Ensure no missing values
if np.isnan(X_array).any():
    print("Data contains NaN values. Handling missing data...")
    # Handle missing data (e.g., imputation)
    X_array = np.nan_to_num(X_array)

In [ ]:
# Encode labels if they are not numerical
if not np.issubdtype(y_array.dtype, np.number):
    print("Encoding labels...")
    le = LabelEncoder()
    y_array = le.fit_transform(y_array)

In [ ]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_array, dtype=torch.float32)
y_tensor = torch.tensor(y_array, dtype=torch.long)  # CrossEntropyLoss expects labels of type Long

In [ ]:
# Create the dataset and DataLoader
full_dataset = TensorDataset(X_tensor, y_tensor)

In [ ]:
# Define split proportions
train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

In [ ]:
# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Lists to store loss and accuracy for plotting
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [ ]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # Validation phase
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for val_X, val_y in val_loader:
            outputs = model(val_X)
            loss = loss_fn(outputs, val_y)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += val_y.size(0)
            val_correct += (predicted == val_y).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
          f'Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')


In [ ]:
# Visualization of Loss and Accuracy

# Plot Loss over epochs
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy over epochs
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()

In [ ]:
# Evaluate on Test Data and Compute Performance Metrics

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for test_X, test_y in test_loader:
        outputs = model(test_X)
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(test_y.cpu().numpy())

# Compute metrics
test_accuracy = accuracy_score(all_labels, all_preds) * 100
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f'Test Accuracy: {test_accuracy:.2f}%')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "base_model.pth")
print("Model saved as 'base_model.pth'")